
#### <center>DVD Rental Co<center>
## <center>Exploratory Data Analysis<center>
### <center>Dive into the legacy data: Migration, Analysis and Key Insights<center>

In [ ]:
import duckdb
with duckdb.connect("data/sakila.duckdb") as conn:
    description = conn.sql("DESC;").df()

In [ ]:
dfs = {}
with duckdb.connect("data/sakila.duckdb") as conn:
    for name in description["name"]:
        if name[0] != "_":
            dfs[name] = conn.sql(f"FROM staging.{name};").df()
            duckdb.register(name, dfs[name])
#dfs.keys()

### Task 1
a) Which movies are longer than 3 hours (180 minutes), show the title and its length?


In [ ]:
over_3h_movies = duckdb.sql("""
        SELECT 
            title, 
            length as "Length (minutes)"
        FROM 
            film
        WHERE 
            "length (minutes)"> 180
        ORDER BY 
            "length (minutes)" DESC;
    """).df()
over_3h_movies.head()

b) Which movies have the word "love" in its title? Show the following columns

In [ ]:
love_in_titles = duckdb.sql("""
        SELECT 
            title, rating, length, description
        FROM 
            film
        WHERE 
            regexp_matches(title, '(?i)\\blove\\b');
    """).df()
love_in_titles

c) Calculate descriptive statistics on the length column, The Manager wants, shortest, average, median and
longest movie length

In [ ]:
film_duration = duckdb.sql(""" 
    SELECT
        MIN(length) as shortest_duration,
        ROUND(AVG(length)) as mean_duration,
        MEDIAN(length) as median_duration, 
        MAX(length) as longest_duration 
    FROM
        film;
    """).df()
film_duration

d) The Manager wants to know the 10 most expensive movies to rent per day

In [ ]:
film_duration = duckdb.sql(""" 
    SELECT 
        title, 
        rental_duration, 
        rental_rate, 
        ROUND(rental_rate/rental_duration, 2) AS "Price/Day"
    FROM
        film
    ORDER BY
        "Price/Day" DESC
    LIMIT 10;
    """).df()
film_duration

e) Which actors have played in most movies? Show the top 10 actors with the number of movies they have
played in.


In [ ]:
top_10_actors = duckdb.sql(""" 
    SELECT
        a.actor_id, 
        a.first_name || ' ' || a.last_name AS actor_name,
        COUNT(fa.film_id) AS movie_count
    FROM
        actor a
    LEFT JOIN 
        film_actor fa ON a.actor_id = fa.actor_id
    GROUP BY
        a.actor_id, a.first_name, a.last_name
    HAVING COUNT(fa.film_id) > 0 
    ORDER BY
        movie_count DESC
    LIMIT 10;
    """).df()
top_10_actors

f) Now it's time for you to choose your own question to explore the sakila database! Write down 3-5
questions you want to answer and then answer them using pandas and duckdb.<br>

Question f1: Which are the top 3 most rented film categories?

In [ ]:
top_3_categories = duckdb.sql("""
    SELECT
        c.name as Category,
        COUNT(r.rental_id) as total_rentals 
    FROM
        category c 
    JOIN film_category fc ON c.category_id = fc.category_id 
    JOIN film f ON fc.film_id = f.film_id 
    JOIN inventory inv ON f.film_id = inv.film_id
    JOIN rental r ON inv.inventory_id = r.inventory_id 
    GROUP BY
        c.name 
    ORDER BY
        total_rentals DESC 
    LIMIT 3;
    """).df()
top_3_categories

Question f2: Who are the churned customers (full name, email, last rental date) to email them a voucher to reactivate them

In [ ]:
check_active = duckdb.sql("DESC main.customer;").df()
check_active
# Note: active is a str/VARCHAR, not Boolean

In [ ]:
# active is a str/VARCHAR with '0' and '1'
check_active2 = duckdb.sql(""" 
    SELECT
        active,
        COUNT(*) as active_count
    FROM
        customer
    GROUP BY active 
    ORDER BY active_count;
    """).df()
check_active2

In [ ]:
churn_cust_list = duckdb.sql(""" 
    SELECT
        cust.customer_id, 
        CONCAT(cust.first_name,' ', cust.last_name) AS customer_name, 
        MIN(r.rental_date) as first_rental, 
        MAX(r.rental_date) as last_rental, 
        cust.email, 
    FROM
        customer cust 
    LEFT JOIN rental r ON cust.customer_id =  r.customer_id 
    WHERE cust.active = '0' 
    GROUP BY ALL 
    ORDER BY customer_name;
    """).df()
churn_cust_list

Question f3: Which store generated the least revenues in 2006?

In [ ]:
# First I need to check how many stores there are.
stores_count = duckdb.sql(""" 
    SELECT 
        COUNT(*) AS store_count 
    FROM store;
    """).df()
stores_count

In [ ]:
# How many business years?
rental_years = duckdb.sql(""" 
    SELECT 
        DISTINCT YEAR(rental_date) as rental_year 
    FROM rental 
    ORDER BY rental_year;
    """).df()
rental_years

In [ ]:
revenue_stores_2006 = duckdb.sql("""
    SELECT 
        s.store_id, 
        CONCAT(ad.address, ', ', ci.city, ', ', cou.country) AS full_address, 
        SUM(p.amount) AS total_revenue, 
    FROM 
        payment p 
    JOIN rental r ON p.rental_id = r.rental_id 
    JOIN inventory i ON r.inventory_id = i.inventory_id 
    JOIN store s ON i.store_id = s.store_id 
    JOIN address ad ON s.address_id = ad.address_id 
    JOIN city ci ON ad.city_id = ci.city_id 
    JOIN country cou ON ci.country_id = cou.country_id 
    WHERE
        EXTRACT(YEAR FROM p.payment_date) = 2006 
    GROUP BY 
        s.store_id, ad.address, ci.city, cou.country  
    ORDER BY
        total_revenue ASC;
    """).df()

revenue_stores_2006   
                                  

In [ ]:
# Checking the above for 2006 as very low revenues
sales_check_2006 = duckdb.sql(""" 
    SELECT 
        s.store_id, 
        EXTRACT(MONTH FROM p.payment_date) AS month, 
        COUNT(p.payment_id) AS rentals_transaction, 
        ROUND(SUM(p.amount), 2) AS revenue, 
    FROM 
        payment p 
    JOIN customer cust ON cust.customer_id = p.customer_id 
    JOIN store s ON cust.store_id = s.store_id 
    WHERE
        EXTRACT(YEAR FROM p.payment_date) = 2006 
    GROUP BY
        s.store_id, EXTRACT(MONTH FROM p.payment_date), 
    ORDER BY
        s.store_id, EXTRACT(MONTH FROM p.payment_date);
    """).df() 
sales_check_2006

In [ ]:
# Checking if transactions numbers match, should be 182 in 2006 
check_monthly_transactions = duckdb.sql("""
    SELECT 
        EXTRACT(YEAR FROM payment_date) AS year, 
        EXTRACT(MONTH FROM payment_date) AS month, 
        COUNT(p.payment_id) AS transactions, 
        ROUND(SUM(p.amount), 2) AS revenue
    FROM
        payment p 
    GROUP BY ALL 
    ORDER BY
        year, month;
    """).df()
check_monthly_transactions

### Task 2 - graphs
a) Who are our top 5 customers by total spend? The Manager wants to know so that they can reward them
with special offers. Create a bar chart showing the top 5 customers by total spend.

In [ ]:
top_5_customers = duckdb.sql(""" 
    SELECT 
        cust.customer_id, 
        cust.first_name || ' ' || cust.last_name AS customer, 
        SUM(p.amount) AS total_spend 
    FROM
        main.customer cust 
    LEFT JOIN main.payment p ON cust.customer_id = p.customer_id 
    GROUP BY
        cust.customer_id, customer 
    ORDER BY
        total_spend DESC 
    LIMIT 5;
    """).df()
top_5_customers

In [ ]:
ax = top_5_customers.plot(
    kind='barh',              
    x='customer',             
    y='total_spend',
    figsize=(6, 2),
    color='steelblue',
    legend=False,
    title='Top 5 Customers by Total Spending'
    )
ax.invert_yaxis()
ax.bar_label(ax.containers[0], fmt='$%.2f') # took from LLM to make values appear 
ax.margins(x=0.15)                          # took from LLM to make values appear   

b) How much money does each film category bring in? Make a bar chart showing total revenue per film
category.


In [ ]:
revenue_per_category = duckdb.sql("""
    SELECT
        c.name AS category_name,
        SUM(p.amount) AS revenues                                              
    FROM
        category c
    LEFT JOIN film_category fc ON c.category_id = fc.category_id
    LEFT JOIN film f ON fc.film_id = f.film_id
    LEFT JOIN inventory inv ON f.film_id = inv.film_id
    LEFT JOIN rental r ON inv.inventory_id = r.inventory_id
    LEFT JOIN payment p ON r.rental_id = p.rental_id
    GROUP BY c.name
    ORDER BY revenues DESC;
""").df()
revenue_per_category

In [ ]:
ax = revenue_per_category.plot(
    kind='barh',              
    x='category_name',             
    y='revenues',
    figsize=(12, 6),
    color='green',
    legend=False,
    title='Revenues per Category'
    )
ax.invert_yaxis()
ax.bar_label(ax.containers[0], fmt='$%.2f') 
ax.margins(x=0.15)               

### Task 3 - BI report
